In [122]:
from main import main,add_arguments
from utils import LoadData, LoadModel, load_data
from train import run_training
import argparse
import numpy as np
import time
import copy
import os
import tempfile

import numpy as np
# pyomo for optimization
import pyomo.environ as pyo

# pytorch for training neural network
import torch.onnx
from torch import nn, optim
from torch.optim.lr_scheduler import StepLR
from torchvision import datasets, transforms

# omlt for interfacing our neural network with pyomo
import onnx
from omlt import OffsetScaling, OmltBlock
from omlt.io.onnx import (
    load_onnx_neural_network_with_bounds,
    write_onnx_model_with_bounds,
    load_onnx_neural_network,
)
from omlt.neuralnet import FullSpaceNNFormulation

In [128]:
args = add_arguments('KKThPINN', 7, 32, 2, 8, 'experiment')

TypeError: add_arguments() takes 0 positional arguments but 6 were given

In [44]:
# def add_arguments(model,model_id,system):    
#     if system == 'membrane':
#         args = argparse.Namespace(
#             input_dim=7,
#             hidden_dim=32,
#             hidden_num=2,
#             z0_dim=8,
#             optimizer='adam',
#             epochs=1000,
#             batch_size=16,
#             lr=1e-4,
#             mu=1,
#             max_subiter=500,
#             eta=0.8,
#             sigma=2,
#             mu_safe=1e+9,
#             dtype=32,
#             dataset_path='/home/andresfel9403/KKThNN/KKThPINN/benchmark_membrane.csv',
#             val_ratio=0.2,
#             job='prediction',
#             runs=10)
#     else:
#         pass

#     args.model = model
#     args.model_id = model_id
#     args.dataset_type = system
        
    
#     if args.model == 'NN':
#         args.loss_type = 'MSE'
#     elif args.model == 'PINN':
#         args.loss_type = 'PINN'
#     elif args.model == 'KKThPINN':
#         args.loss_type = 'MSE'
#     elif args.model == 'AugLagNN':
#         args.loss_type = 'MSE'
#     elif args.model == 'ECNN':
#         args.loss_type = 'MSE'
#     return args

After defining the arguments depending on the number of inputs, hidden and output layers, we can train the model selecting the Neural Network model (NN or KKThPINN)

In [82]:
args = add_arguments('KKThPINN','jum','membrane')
dataset, scaling = load_data(args.dataset_path)
inputs = dataset[:,:7]
outputs = dataset[:,7:]

data = LoadData(args)
model = LoadModel(args, data)
PATH = '/home/andresfel9403/KKThNN/models/membrane/KKThPINN/0.2/None_0.2_9.pth'
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['state_dict'])
model.eval()



type of A: torch.float32, type of B: torch.float32, type of b: torch.float32
train set size: 614, val set size: 204, test set size: 204


/tmp/ipykernel_270476/3524242555.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(PATH)


NNOPT(
  (layers): ModuleList(
    (0): Linear(in_features=7, out_features=32, bias=True)
    (1): Linear(in_features=32, out_features=32, bias=True)
    (2): Linear(in_features=32, out_features=8, bias=True)
  )
  (fc_fixed1): Linear(in_features=8, out_features=8, bias=False)
  (fc_fixed2): Linear(in_features=7, out_features=8, bias=True)
)

In [46]:
x_factor = scaling.scale_[:7]
y_factor = scaling.scale_[7:]

print(x_factor)

[1.46831994e+01 1.04111445e+01 1.42231966e+01 9.99584502e+01
 7.26638589e+02 2.69926289e+06 1.76773269e+02]


In [47]:
scaler = scaler = OffsetScaling(
    offset_inputs={i: 0 for i in range(len(x_factor))},
    factor_inputs={i: x_factor[i] for i in range(len(x_factor))},
    offset_outputs={i: 0 for i in range(len(y_factor))},
    factor_outputs={i: y_factor[i] for i in range(len(y_factor))},
)

In [48]:
print(1*x_factor[0])

14.68319944


In [49]:
x_dummy = torch.from_numpy(inputs[0]).float()
ub = np.max(inputs, 0)
lb = np.min(inputs, 0)

scaled_input_bounds = {i: (lb[i], ub[i]) for i in range(len(inputs[0]))}
print(scaled_input_bounds)

{0: (np.float64(0.002303625455624813), np.float64(1.0)), 1: (np.float64(0.00045880834803283796), np.float64(1.0)), 2: (np.float64(0.0022512226291679363), np.float64(1.0)), 3: (np.float64(0.10018982303101381), np.float64(1.0)), 4: (np.float64(0.25788606628182886), np.float64(1.0)), 5: (np.float64(0.4816711270961038), np.float64(1.0)), 6: (np.float64(0.056910004685567034), np.float64(1.0))}


In [50]:
for key, value in data.items() :
            print (key, value)

train_loader <torch.utils.data.dataloader.DataLoader object at 0x7fcb855c3f20>
val_loader <torch.utils.data.dataloader.DataLoader object at 0x7fcb855c3ec0>
test_loader <torch.utils.data.dataloader.DataLoader object at 0x7fcb85628f50>
dataset <utils.Data_membrane object at 0x7fcb856d7500>
A tensor([[14.6832, 10.4111, 14.2232, 99.9585,  0.0000,  0.0000,  0.0000]])
B tensor([[ -3.3498, -11.7057,  -5.5663, -14.7299, -99.9585,   0.0000,   0.0000,
           0.0000]])
b tensor([[0.]])
constrained_indexes [0, 1, 2, 3, 4]
unconstrained_indexes [5, 6, 7]
B_dep tensor([[-3.3498]])
B_indep tensor([[-11.7057,  -5.5663, -14.7299, -99.9585,   0.0000,   0.0000,   0.0000]])


In [51]:
print(model)

NN(
  (layers): ModuleList(
    (0): Linear(in_features=7, out_features=32, bias=True)
    (1): Linear(in_features=32, out_features=32, bias=True)
    (2): Linear(in_features=32, out_features=8, bias=True)
  )
)


In [68]:
with tempfile.NamedTemporaryFile(suffix=".onnx", delete=False) as f:
    # export neural network to ONNX
    torch.onnx.export(
        model,
        x_dummy,
        f,
        input_names=["input"],
        dynamo = False
    )
    # write ONNX model and its bounds using OMLT
    write_onnx_model_with_bounds(f.name, None, scaled_input_bounds)
    #onnx_model = onnx.load(f.name)
    # load the network definition from the ONNX model
    network_definition = load_onnx_neural_network_with_bounds(f.name)
    #network_definition = load_onnx_neural_network(onnx = onnx_model) #scaling_object=scaler, input_bounds= scaled_input_bounds) 

In [69]:
for layer_id, layer in enumerate(network_definition.layers):
    print(f"{layer_id}\t{layer}\t{layer.activation}")

0	InputLayer(input_size=[7], output_size=[7])	linear
1	DenseLayer(input_size=[7], output_size=[32])	relu
2	DenseLayer(input_size=[32], output_size=[32])	relu
3	DenseLayer(input_size=[32], output_size=[8])	linear


In [70]:
formulation = FullSpaceNNFormulation(network_definition)

In [71]:
m = pyo.ConcreteModel()

# create an OMLT block for the neural network and build its formulation
m.membrane = OmltBlock()
m.membrane.build_formulation(formulation)



WARNING (W1002): Setting Var 'membrane.scaled_inputs[0]' to a numeric value
`0` outside the bounds (0.002303625455624813, 1.0).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'membrane.scaled_inputs[0]' to a numeric value
`0` outside the bounds (0.002303625455624813, 1.0).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'membrane.scaled_inputs[1]' to a numeric value
`0` outside the bounds (0.00045880834803283796, 1.0).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'membrane.scaled_inputs[2]' to a numeric value
`0` outside the bounds (0.0022512226291679363, 1.0).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'membrane.scaled_inputs[3]' to a numeric value
`0` outside the bounds (0.10018982303101381, 1.0).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING 

In [72]:
m.obj = pyo.Objective(expr=m.membrane.outputs[1], sense=pyo.maximize)

In [73]:
solver = pyo.SolverFactory("cbc")
status = solver.solve(m, tee=True)

Welcome to the CBC MILP Solver 
Version: 2.10.7 
Build Date: Feb 14 2022 

command line - /usr/bin/cbc -printingOptions all -import /tmp/tmpal2j_1gh.pyomo.lp -stat=1 -solve -solu /tmp/tmpal2j_1gh.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
 CoinLpIO::readLp(): Maximization problem reformulated as minimization
Coin0009I Switching back to maximization to get correct duals etc
Presolve 250 (-116) rows, 188 (-57) columns and 1880 (-284) elements
Statistics for presolved model
Original problem has 64 integers (64 of which binary)
Presolved problem has 56 integers (56 of which binary)
==== 187 zero objective 2 different
187 variables have objective of -0
1 variables have objective of 1
==== absolute objective values 2 different
187 variables have objective of 0
1 variables have objective of 1
==== for integers 56 zero objective 1 different
56 variables have objective of -0
==== for integers absolute objective values 1 different
56 variables have obje

In [80]:
print("FA:", pyo.value(m.membrane.inputs[6]))

FA: 1.0
